# Linear Algebra

## Chapter 1: Numerical vector space

$x \in A$: x is an **ELEMENT** of A  
$A \subseteq B$: A is **SUBSET** of B  
$\varnothing, \emptyset$: Empty set  
$A \cap B$: Intersection  
$A \cup B$: Union  
$A^c, \overline{A} = \{x \in U \mid x \notin A\}$: Complement  
$A \setminus B, A - B = \{x \mid x \in A \cap x \notin B\}$: Difference  
$\forall x$: All x  
$\exists x$: A x  

## Basic Calculation

### Preparing

In [ ]:
def row(A, n):
    return A[n]

In [ ]:
def col(A, n):
    return [a[n] for a in A]

In [ ]:
def prod(array):
    product = 1
    for a in array:
        product *= a
    return product

In [ ]:
def linear_combination(a, b):
    return sum(map(prod, zip(a, b)))

### Calculator(Sum, Diff, Product etc.)

In [ ]:
def add(A, B):
    if isinstance(B, (int, float)):
        A, B = B, A
    if isinstance(A, (int, float)):
        return [[A + x for x in b] for b in B]
    return [[x + y for x, y in zip(a, b)] for a, b in zip(A, B)]

In [ ]:
def sub(A, B):
    if isinstance(B, (int, float)):
        return add(A, -B)
    return [[x - y for x, y in zip(a, b)] for a, b in zip(A, B)]

In [ ]:
def multi(A, B):
    if isinstance(B, (int, float)):
        A, B = B, A
    if isinstance(A, (int, float)):
        return [[A * x for x in b] for b in B]
    raise ArithmeticError('Not calculatable type')

In [ ]:
def divide(A, b):
    if not isinstance(b, (int, float)):
        raise ArithmeticError()
    return 

In [ ]:
def dot(A, B):
    if len(row(A, 0)) != len(col(B, 0)):
        raise ArithmeticError('Different dimmension arrays are not calculatable')
    return [[linear_combination(row(A, i), col(B, j)) for j in range(len(col(B, 0)))] for i in range(len(row(A, 0)))]

In [ ]:
def transpose(A):
    return [col(A, i) for i in range(len(row(A, 0)))]
t = transpose

In [ ]:
def eye(n):
    return [[int(i == j) for i in range(n)] for j in range(n)]

In [ ]:
def zeros(n, m=0):
    if m > 0:
        return [[0 for i in range(m)] for j in range(n)]
    return [[0 for i in range(n)] for j in range(n)]

### Operation Checking

#### Scalar Calculation

In [ ]:
A = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [ ]:
add(A, 10)

In [ ]:
sub(A, 1)

In [ ]:
multi(A, 10)

In [ ]:
transpose(A)

In [ ]:
t(A)

In [ ]:
eye(3)

In [ ]:
zeros(3)

#### Matrix Calculation

In [ ]:
A = [[1, 2], [3, 4]]
B = [[5, 6], [7, 8]]

In [ ]:
add(A, B)

In [ ]:
sub(B, A)

In [ ]:
add(B, multi(-1, A))

In [ ]:
dot(A, B)

## Chapter2: Sweep Method

In [ ]:
A = [[ 9, 2, 13],
     [-3, 4,  5]]

B = [[1, 1, 1, 2],
     [1, 2, 3, 2],
     [1, 3, 3, 0]]

In [ ]:
def coef_sweeping(a, n):
    if not a[n]:
        raise ZeroDivisionError()
    return list(map(lambda x: x / a[n], a))

In [ ]:
def sweep(A):
    n = len(A)
    for i in range(n):
        pivot = A[i][i]
        A[i] = coef_sweeping(A[i], povot)
        for j in range(i + 1, n):
            p = A[j][i]
            A[j] = sub()
        

In [ ]:
sweep(A)